In [ ]:
import numpy as np
import skimage.io
import skimage.transform
import matplotlib.pyplot as plt
import pandas as pd
import shutil, pickle, os, random, matplotlib
from PIL import Image, ImageEnhance
from sklearn.model_selection import train_test_split
import torch
import torch.nn.functional as F
from torch import nn
import configparser


# <strong>Main</strong>


## Data 
The data is converted into RGB tensors with skimage.io.imread()

<strong>Notice:</strong> pip install scikit-image 

### (function) Pickling data 

In [ ]:
def pickle_data(file, writeColumns=None):
    """
    Read/Write pickle training/testing data, models to avoid
    loading data again (time consuming)
    
    ---Params---

    file: path to pickle file

    writeColumns (array): variables to be saved to pickle file

    """
    if writeColumns is None:
        with open(file, mode="rb") as f:
            dataset = pickle.load(f)
            return tuple(map(lambda col: dataset[col], ['images', 'labels'])) # lambda(col) where columns are the inputs
    else:
        with open(file, mode="wb") as f:
            dataset = pickle.dump({"images": writeColumns[0], "labels": writeColumns[1]}, f)
            print("Data is saved in", file)
# lambda function: https://www.youtube.com/watch?v=BcbVe1r2CYc


### (function) Label the test dataset


In [ ]:
def label_test(src, csv_file, labeled_test_dir, NoOfCategories):
    """
    This function creates named folders corresponding to 43 categories
    and move the test images to these folders

    `csv_file` and `labeled_test_dir` should have already been in src directly 
    (create a blank folder to store labeled images)

    """
    # Remove the existing folders in the labeled test directory if there is any
    for filename in os.listdir(labeled_test_dir):
        file_path = os.path.join(labeled_test_dir, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

    csv_dir = os.path.join(src, csv_file)
    SubTestDir = [os.path.join(labeled_test_dir, str(d)) for d in range(NoOfCategories)]
    
    # Create label folders
    [os.mkdir(test_d) for test_d in SubTestDir]

    testImageDir = pd.read_csv(csv_dir)['Path']
    testImageLabel = pd.read_csv(csv_dir)['ClassId']
    for idx in range(len(testImageLabel)):
        label = testImageLabel[idx]
        shutil.copy(os.path.join(src, testImageDir[idx]), SubTestDir[label])

# labeled_test_dir = "D:\Programming Files\Python Files\Deep learning - traffic signs\German\LabeledTest"
labeled_test_dir = r"C:\Users\lemin03\Documents\traffic_class\TrafficSignRec\German\Labeled_test" # BH   
# src = "D:\Programming Files\Python Files\Deep learning - traffic signs\German" # BH
src = r"C:\Users\lemin03\Documents\traffic_class\TrafficSignRec\German"
csv_file = "Test.csv"
NoOfCats = 43

label_test(src, csv_file, labeled_test_dir, NoOfCats)


### (function) Load 

In [ ]:
def load_data(data_dir):
    """Loads a data set and returns two lists:
    
    images: a list of Numpy arrays, each representing an image.
    labels: a list of numbers that represent the images labels.
    """
    # Get all subdirectories of data_dir. Each represents a label.
    directories = [d for d in os.listdir(data_dir) 
                   if os.path.isdir(os.path.join(data_dir, d))]
    # Loop through the label directories and collect the data in
    # two lists, labels and images.
    labels = []
    images = []
    for d in directories:
        # label_dir contains 61 catefories paths
        label_dir = os.path.join(data_dir, d)

        # list subdirectories within each of the 61 categories
        file_names = [os.path.join(label_dir, f) 
                      for f in os.listdir(label_dir) if f.endswith(".png")]
        # For each label, load it's images and add them to the images list.
        # And add the label number (i.e. directory name) to the labels list.
        for f in file_names:
            images.append(skimage.io.imread(f))
            labels.append(int(d))
    return images, labels

# ROOT_PATH = "D:\Programming Files\Python Files\Deep learning - traffic signs\German"
ROOT_PATH = r"C:\Users\lemin03\Documents\traffic_class\TrafficSignRec\German"

### Last execution 10:01PM 20/5/23
!!!: resized images are already normalized to [0,1] format

In [ ]:
# Load training dataset.
train_data_dir = os.path.join(ROOT_PATH, "Train")
first_train_images, first_train_labels = load_data(train_data_dir)
pickle_data(file = "primary_train_dataset", writeColumns = [first_train_images, first_train_labels] )

In [ ]:
# Apply constant resolution among train images and pickle them 
train_images, train_labels  = pickle_data(file = 'primary_train_dataset')

train_images = [ skimage.transform.resize(train_image, (32, 32), mode = "constant") 
                            for train_image in train_images ]

train_images = np.stack(train_images, axis = 0)

pickle_data(file = "primary32_train_dataset", writeColumns = [train_images, train_labels])

In [ ]:
# Load the test dataset
test_data_dir = os.path.join(ROOT_PATH, "Labeled_test")
first_test_images, first_test_labels = load_data(test_data_dir)
test_images, val_images, test_labels, val_labels = train_test_split(first_test_images, first_test_labels, 
                                                                        test_size=0.36, random_state=0)
pickle_data(file = "primary_test_dataset", writeColumns = [test_images, test_labels])
pickle_data(file = "primary_val_dataset", writeColumns = [val_images, val_labels])

In [ ]:
# Apply constant resolution among test,val images and pickle them 
test_images, test_labels  = pickle_data(file = 'primary_test_dataset')
val_images, val_labels  = pickle_data(file = 'primary_val_dataset')

test_images = [ skimage.transform.resize(test_image, (32, 32), mode = "constant") 
                            for test_image in test_images ]
val_images = [ skimage.transform.resize(val_image, (32, 32), mode = "constant") 
                            for val_image in val_images ]

test_images = np.stack(test_images, axis = 0)
val_images = np.stack(val_images, axis = 0)

pickle_data(file = "primary32_test_dataset", writeColumns = [test_images, test_labels])
pickle_data(file = "primary32_val_dataset", writeColumns = [val_images, val_labels])

### Load pickled data

In [ ]:
# Program starts here if pickle folders are not updated
train_images, train_labels  = pickle_data(file = './Data/primary32_train_dataset')
test_images, test_labels  = pickle_data(file = './Data/primary32_test_dataset')
val_images, val_labels  = pickle_data(file = './Data/primary32_val_dataset')

In [ ]:
print(train_images.shape)

### (function) Display images


In [ ]:
def display_images(images, labels, category=False, greyScale=False):
    """
    Display the first image of each label.
    
    category: set to True when only images within a category are displayed
    greyScale: set to True to display images in grey scale
    """

    if category:
        i = 1
        startIndex = labels.index(category)
        catImages = images[startIndex:(startIndex + labels.count(category))] #catImage = categoryImage
        
        plt.figure(figsize=(15, 15))
        for catImage in catImages[:24]:
            plt.subplot(8, 8, i)
            plt.xticks([])
            plt.yticks([])
            plt.imshow(catImage)
            i += 1 
    else:
        unique_labels = set(labels) # Create a list contains only the labels (non-iterative)
    
        #Example: a = [1, 1, 1, 2, 2, 3]
        #set(a) >> {1,2,3}

        plt.figure(figsize=(15, 15))
        i = 1
        for label in unique_labels:
            image = images[labels.index(label)] # Pick the first image for each label.

        # object.index(element) returns the index of the element specified when it's first encountered
        # Example: a = [1, 1, 1, 3, 2, 2, 3]
        # a.index(3) = 3

            plt.subplot(8, 8, i)  # A grid of 8 rows x 8 columns
            plt.xticks([])
            plt.yticks([])
            plt.title("Label {0} ({1})".format(label, labels.count(label))) # sign category and the # of its samples
            if greyScale is False:
                plt.imshow(image)
            else:
                plt.imshow(image, cmap=plt.cm.binary)
            i += 1
    plt.show()

### (function) Convert to grey scale, improve contrast

In [ ]:
from skimage import exposure

def preprocess_images(images):
    """
        - Convert RGB images to grey scale 
        - Normalize pixels to 0-1,
        - Improve the contrast with adaptive histogram localization
    """
     
    # Conver RGB -> grey scale
    images = 0.299 * images[:, :, :, 0] + 0.587 * images[:, :, :, 1] + 0.114 * images[:, :, :, 2]
    # Improve the contrast
    images = exposure.equalize_adapthist(images)

    # Add ONE 3-D channel for grey scale
    images = images.reshape(images.shape + (1,)) 

    return images

In [ ]:
a = train_images[0]
# print(a.shape)
a = 0.299 * a[:, :, 0] + 0.587 * a[:, :, 1] + 0.114 * a[:, :, 2]
print(a.shape)
a = exposure.equalize_adapthist(a)
a = a.reshape(a.shape + (1,)) 
plt.imshow(a,cmap=plt.cm.binary)


In [ ]:
# Display all images 
display_images(preprocess_images(train_images, improveCon=True), train_labels, greyScale=True)

In [ ]:
# Display all images 
display_images(preprocess_images(train_images), train_labels)

### Classes distribution

In [ ]:
fig, ax = plt.subplots()
n_classes = len(set(train_labels))
values, bins, patches = ax.hist(train_labels, n_classes)
ax.set_xlabel("Classes")
ax.set_ylabel("Number of images")

## Data Augmentation

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

X_train = train_images

train_datagen = ImageDataGenerator()
inference_datagen = ImageDataGenerator()
train_datagen_augmented = ImageDataGenerator(
    rotation_range=20,
    shear_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)
train_datagen.fit(X_train)
train_datagen_augmented.fit(X_train)
fig = plt.figure()
n = 0
graph_size = 3

for x_batch, y_batch in train_datagen_augmented.flow(X_train, train_labels, batch_size=4):
    a=fig.add_subplot(graph_size, graph_size, n+1)
    greyBatch = preprocess_images(x_batch)
    print(greyBatch[0].shape)
    imgplot = plt.imshow(greyBatch[0])
    plt.xticks([])
    plt.yticks([])
    plt.title("Label:{}".format(y_batch[0]))
    n = n + 1
    if n > 8:
        break
# plt.show()

## Model


### Capsule layer attributes
TODO:
- build a simple conv layer: DONE (first conv layer with relu)
- build capsule's functions (squash, routing)
- build primary and secondary capsnet
- fully connected layer for reconstruction
- loss functions
- add visualizations to track the changes in prior logits, training loss

In [ ]:
# Currently this network only supports grey scale imgage due to fully connected reconstruction layer

from sklearn.utils import shuffle
from keras.preprocessing.image import ImageDataGenerator

config = configparser.ConfigParser()
config.read(
    'D:\Programming Files\Python Files\Deep learning - traffic signs\src\capsnet_config.ini'
    )

# General info
num_class = config['network']['num_class']
img_channels = config['network']['image_channels']
batch_size = config['network']['batch_size']

# Primary caps
primary_num_caps = config['primary_caps']['num_caps']
primary_channels = config['primary_caps']['channels']

# Digit caps
digit_num_caps = num_class
digit_channels = config['digit_caps']['channels']
num_iterations = config['network']['num_routing_iter']

# Loss hyper params
m_plus = config['loss']['m_plus']
m_minus = config['loss']['m_minus']
lmbd = config['loss']['lambda']
regularization_factor = config['loss']['regularization_factor']

shuffeled_train_images, shuffeled_train_labels = shuffle(train_images, train_labels, random_state=0)
demo_train_images = shuffeled_train_images[0:1]
demo_train_labels = shuffeled_train_labels[0:1]
X_train = demo_train_images

train_datagen = ImageDataGenerator()
inference_datagen = ImageDataGenerator()
train_datagen_augmented = ImageDataGenerator(
    rotation_range=20,
    shear_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)
train_datagen.fit(X_train)
train_datagen_augmented.fit(X_train)

def squash(vector, axis=-1 ,epsilon=1e-7, squash=True):
        """
        normalize the length of the vector by 1
        `vector`: the muliplication of coupling coefs and prediction vectors sum [ c(ij)u^(j|i) ]
        `axis`: the axis that would not be reduced
        'epsilon`: a workaround to prevent devision by zero
        """
        squared_norm = torch.sum(torch.square(vector), dim=axis, 
                                keepdim=True)
        safe_norm = torch.sqrt(squared_norm + epsilon)

        if squash:
            squash_factor = squared_norm / (1. + squared_norm)
            unit_vector = vector / safe_norm
            return squash_factor * unit_vector
        else:
            return safe_norm


class capsLayers(nn.Module):

    """ 
    Args:
    :param num_conv: number of filters/convolutional unit per capsule (dimension of a capsule)
    :param num_capsules: number of primary/digit caps
    :param num_routing_nodes: number of possible u(i), 
                            set to -1 if it's not a secondary capsule layer
    :param in_channels: output convolutional layers of the prev layer
    :param out_channels: output convolutional layers of the current layer
    """
    def __init__(self, num_capsules: int, in_channels: int, out_channels: int, 
                 kernel_size=None, stride=None, num_routing_nodes=None ,num_iterations=None):
        super(capsLayers, self).__init__()
        self.num_capsules = num_capsules
        
        self.num_iterations = num_iterations

        if num_routing_nodes is not None:
            self.num_routing_nodes = num_routing_nodes
            self.weights = nn.Parameter(torch.randn(
                                        num_routing_nodes, num_capsules, out_channels, in_channels))
            self.b = nn.Parameter(torch.zeros(
                                    num_routing_nodes, num_capsules, 1, 1))

        else:
            self.primary_caps = nn.ModuleList(nn.Conv2d(in_channels, out_channels, kernel_size, stride) 
                                                    for _ in range(num_capsules))

    def forward(self, inputs):
        """
        Feed foward function for non-reconstruction layer
        :param inputs: 
            for the primary caps, the inputs are convolutional layer pixels
            for digit caps, the inputs are n-D vectors from a primary cap
                where n is the # of filters for one capsule  
            Required Paramteters:
            prior_logits(b) 
            primary layer prediction (requires u-layer 1 ouput, Weights)
        """
        if self.num_routing_nodes is not None:
            weights = self.weights[None, :, :, :].tile(inputs.size(0), 1, 1, 1, 1)
            b_ij = self.b[None, :, :, :].tile(inputs.size(0), 1, 1, 1, 1)
            inputs = inputs.tile(1, 1, self.num_capsules, 1, 1)
            
            # u_hat = [batch, num_routing_nodes, # digit_caps, digit_caps_dims, 1]
            u_hat = weights @ inputs 

            for i in range(self.num_iterations):
                c_ij = F.softmax(b_ij, dim=2)
                outputs = self.squash((c_ij*u_hat).sum(dim=1, keepdim=True))

                if i < self.num_iterations - 1 :
                    # v_j OR outputs = [batch, 1 -> num_routing_nodes, num_digit_caps, digit_caps_dims, 1 )]
                    b_ij +=  (b_ij + torch.transpose(u_hat, 3, 4) @ outputs.tile(1, self.num_routing_nodes, 1, 1, 1))

        else:
            outputs = [
                capsule(inputs)[:, None, :, :, :].permute(0, 1, 3, 4, 2) for capsule in self.primary_caps]
            outputs = torch.cat(outputs, dim=1)
            # u(i) = [batch, num_prim_caps*prim_caps_2D_size, prim_caps_output_dimension]
            outputs = outputs.view(outputs.size(0), -1, outputs.size(4)) 
            outputs = self.squash(outputs)[:, :, None, :, None]
        
        # outputs = [batch, 1, num_digit_caps/num_class, digit_caps_dims, 1 )]
        return outputs            


class CapsNet(nn.Module):
    """
        This class contains the full CapsNet architecture:
        
        Convolutional -> primary capsules -> digit capsules -> (3) fully connected
    """
    def __init__(self):
        """
        Params: 
        `inputs`: a 4D tensor (grey scale or RGB)
        """
        super(CapsNet, self).__init__()

        self.conv_1 = nn.Conv2d(in_channels=img_channels, out_channels=256, #TODO: !!! `in_channels` depends on grey scale/RGB image
                                kernel_size=9, stride=1)
        self.primary_caps = capsLayers(primary_num_caps, 256, primary_channels, 
                                        kernel_size=5, stride=2)
        self.digit_caps = capsLayers(digit_num_caps, primary_channels, digit_channels, 
                                     num_routing_nodes=10*10*primary_num_caps, num_iterations=num_iterations)
        self.grey_scale_decoder = nn.Sequential(
                    nn.Linear(digit_channels*digit_num_caps, 576, device='cuda'),
                    nn.ReLU(),
                    nn.Linear(576, 1600, device='cuda'),
                    nn.ReLU(),
                    nn.Linear(1600, 1024, device='cuda'),
                    nn.Sigmoid()
        )

    def forward(self, images, labels=None): # labels should be applied `one_hot` function
                                            # before being used in this forward method
        conv_1_ouputs = F.relu(self.conv_1(images))
        primary_caps_outputs =  self.primary_caps(conv_1_ouputs)
        digit_caps_outputs = self.digit_caps(primary_caps_outputs).squeeze(1)
        
        assert list(digit_caps_outputs.size()) == [images.size()[0], num_class, digit_channels, 1]
               
        v_norm = squash(digit_caps_outputs, axis=-2, squash=False)
        v_prob = F.softmax(v_norm, dim=1)

        self.img = images
        self.v_norm = v_norm

        # Masking
        if labels is None:
            _, idx = torch.max(v_prob, dim=1)
            labels = torch.eye(num_class).index_select(dim=0, index = idx.squeeze())
        
        masked_v = labels[:, :, None, None] * digit_caps_outputs
        
        # Reconstruction
        if images.size()[-1] == 1:
            reconstructed_img = self.grey_scale_decoder(masked_v)

        else:
            # RGB decoder
            pass
    
        return reconstructed_img 
        

    def total_loss(self, reconstructed_img, labels): # TODO: convert these inputs to self. in def forward()
        # Margin loss
        max_1 = F.relu(m_plus - self.v_norm)
        max_2 = F.relu(self.v_norm - m_minus)
        T_k = labels[:, :, None, None]
        
        L_k = T_k * torch.square(max_1) + lmbd * (1 - T_k) * torch.square(max_2)

        assert L_k.size() == self.v_norm.size()
        margin_loss = L_k.sum(dim=1).mean()
        
        # Reconstruction loss
        reconstruction_loss_obj = nn.MSELoss()

        # original_img = [batch size, flatten image (pixels are flatten into arrays)]
        original_img = self.img.view(batch_size, -1)
        reconstruction_loss = reconstruction_loss_obj(reconstructed_img, original_img)
        
        total_loss = margin_loss + regularization_factor * reconstruction_loss

        return total_loss
   
def main():
    fig = plt.figure()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(device, "will be used")
    # model = CapsNet()
    n = 0
    for x_batch, y_batch in train_datagen_augmented.flow(X_train, demo_train_labels, batch_size=1):
        greyBatch = torch.tensor(preprocess_images(x_batch))
        # greyBatch = torch.tensor(x_batch)
        greyBatch = torch.tensor(greyBatch.permute(0, 3, 1, 2)) # [batch_size, channels, height, width]
        # convolved_image = model.forward(greyBatch)
        print(greyBatch.size())
        # # plt.imshow(convolved_image.data.numpy())
        n += 1
        if n >2:
            break
main()


In [ ]:
a = torch.tensor(512)
b = torch.tensor(40)
print(torch.sqrt(a))
print(b.pow(2))



In [ ]:
import tensorflow as tf
x = tf.constant([1.8, 2.2], dtype=tf.float32)
print(tf.cast(x, tf.int32))

Demo (based on German dataset but matrix sizes are chosen on purpose to match MNIST)

In [ ]:
demo_conv = nn.Conv2d(in_channels=3, out_channels=256, kernel_size=13, stride=1) #kernel = 9, stride = 1 for 10x10
primary_caps =  nn.ModuleList(nn.Conv2d(256, 8, 9, 2) for _ in range(32))#kernel = 5 or 6, stride = 2 for 10x10

def squash(vector, axis=-1, epsilon=1e-7, squash=True):
        """
        normalize the length of the vector by 1
        `vector`: the muliplication of coupling coefs and prediction vectors sum [ c(ij)u^(j|i) ]
        `axis`: the axis that would not be reduced
        'epsilon`: a workaround to prevent devision by zero
        """
        squaredNorm = torch.sum(torch.square(vector), dim=axis, 
                                keepdim=True)
        safeNorm = torch.sqrt(squaredNorm + epsilon)
        
        if squash:
                squashFactor = squaredNorm / (1. + squaredNorm)
                unitVector = vector / safeNorm
                return squashFactor * unitVector
        else:
                return squaredNorm

def test_routing(inputs, num_capsules, num_routing_nodes, num_iterations ):
        b = nn.Parameter(torch.zeros(num_routing_nodes, num_capsules, 1, 1))
        weights = nn.Parameter(torch.rand(num_routing_nodes, num_capsules, 16, 8))

        weights = weights[None, :, :, :].tile(inputs.size(0), 1, 1, 1, 1)
        b_ij = b[None, :, :, :].tile(inputs.size(0), 1, 1, 1, 1)
        inputs = inputs.tile(1, 1, num_capsules, 1, 1)
  
        # u_hat = [batch, num_routing_nodes, # digit_caps, digit_caps_dims, 1]
        u_hat = torch.matmul(weights, inputs)
    
        for i in range(num_iterations):
                c_ij = F.softmax(b_ij, dim=2)
                v_j = squash((c_ij*u_hat).sum(dim=1, keepdim=True))
                
                if i < num_iterations - 1 :
                        # v_j = [batch, 1 -> num_routing_nodes, num_digit_caps, digit_caps_dims, 1 )]
                        b_ij +=  (b_ij + torch.transpose(u_hat, 3, 4) @ v_j.tile(1, num_routing_nodes, 1, 1, 1))

        return v_j 

def main():
        a = torch.Tensor(train_images[0]) # RGB images
        b = torch.Tensor(train_images[1])
        c = torch.stack((a,b), dim=0) # create a batch of 2 images
        d = c.view(c.size(0), -1)
        e = torch.ones(d.size())

        err = d - e
        err_square = torch.square(err)
        # print(err_square.size())
        print(err_square.sum(dim=1)) #.mean())
        
        loss1 = nn.MSELoss(reduction='sum')
        loss2 = nn.MSELoss(reduction='mean')

        print(loss1(d, e), loss2(d, e))

        num_categories = 10
        conv_output = demo_conv(c.permute(0, 3, 1, 2))
        # ouputs -> list: len(list) = num_caps, outputs elements -> tensor: size (1,8,10,10)
        caps_output = [
                (cap(conv_output))[:, None, :, :, :].permute(0, 1, 3, 4, 2) for cap in primary_caps]
        output = torch.cat(caps_output, dim=1)
        output = output.view(output.size(0), -1, output.size(4))
        output = squash(output)[:, :, None, :, None]
        # print(conv_output.size(), output.size())

        #routing
        v_j = test_routing(output, 10, 1152, 3).squeeze(1)
        v_j_norm = squash(v_j, axis=-2, squash=False)
        v_softmax = F.softmax(v_j_norm, dim=1)
        # if y is None:
        v_active, idx = torch.max(v_softmax, dim=1)
        y = torch.eye(10).index_select(dim=0, index = idx.squeeze())
        
        masked_v = y[:, :, None, None] * v_j
        # print(masked_v.size())
        print(v_j_norm.size())
        assert list(v_j.size()) == [c.size()[0], 10, 16, 1]
   
main()

In [ ]:
a = F.one_hot(torch.tensor(test_labels), 43)
print(set(test_labels))

# <strong>Miscellaneous</strong>

## Super() examples
use super to access the characteristics of other classes
Ex:
super().__init__(mammalName) is equivalent to Class1.__init_(self, mammalName)

### Example 1

In [ ]:
class Animal(object):
  def __init__(self, Animal):
    print(Animal, 'is an animal.')

class Mammal(Animal):
  def __init__(self, mammalName):
    print(mammalName, 'is a warm-blooded animal.')
    super().__init__(mammalName)

class NonMarineMammal(Mammal):
  def __init__(self, NonMarineMammal):
    print(NonMarineMammal, "can't swim.")
    super().__init__(NonMarineMammal)
   
class NonWingedMammal(Mammal):
  def __init__(self, NonWingedMammal):
    print(NonWingedMammal, "can't fly.")
    super().__init__(NonWingedMammal)

class Dog(NonMarineMammal, NonWingedMammal):
  def __init__(self):
    print('Dog has 4 legs.')
    super().__init__('Dog')
    
d = Dog()
print(d)
# bat = NonMarineMammal('Bat')

In [ ]:
class Parent:
  def __init__(self, *txt):
    # *args is not a must have input
    self.message = txt

  def printmessage(self):
    print(self.message)

  @staticmethod
  def printmessage2(text):
    print(text)


class Child(Parent):
  def __init__(self, txt: str, num):
    self.num = num
    super(Child, self).__init__()
    self.printmessage()
  
  def call_static(self, msg):
    self.printmessage2(msg)

x = Child("Hello, and welcome!", 2)

x.printmessage2("hi static") # another way to call Parent's method

### Example 2

In [ ]:
class Rectangle(object):
    def __init__(self, length, width):
        self.length = length
        self.width = width

    def area(self):
        return self.length * self.length

    def perimeter(self):
        return 2 * self.length + 2 * self.width

# Here we declare that the Square class inherits from the Rectangle class
class Square(Rectangle):
    def __init__(self, length_sqr):
        super().__init__(length_sqr, length_sqr)   # length_sqr = length and width of class Rectangle
Square(5).area()

### PIL module

In [ ]:

a = np.matrix('250 60 143; 90 100 40; 120 150 200')
im = Image.fromarray(a) # create a n image object as arrays
plt.imshow(im)

### Override

In [ ]:
class Employee:
    def __init__(self, name, base_pay):
        self.name = name
        self.base_pay = base_pay

    def get_pay(self):
        return self.base_pay


class SalesEmployee(Employee):
    def __init__(self, name, base_pay, sales_incentive):
        self.name = name
        self.base_pay = base_pay
        self.sales_incentive = sales_incentive

    def get_pay(self):
        return self.base_pay + self.sales_incentive


if __name__ == '__main__':
    john = SalesEmployee('John', 5000, 1500)
    print(john.get_pay())

    jane = Employee('Jane', 5000)
    print(jane.get_pay())

### Dimension visualization

In [ ]:
input_caps = 3
input_dims = 3
output_caps = 2
output_dims = 2
a = torch.Tensor(input_caps, input_dims , output_caps * output_dims)
b = torch.Tensor([ [[10], [30]], [[50], [70]] ])
print(b.shape)
plt.imshow(b, cmap =plt.cm.binary)
print(a, b)

!!! 4D requires diff inputs compared to 3D

In [ ]:
a = torch.randn(2,2,3)
b = torch.randn(2,2,2,3)
print(a)
print('---------------')
print(b)

### Convolutional operation

In [ ]:
# Python program to perform 2D convolution operation on an image
# Import the required libraries

'''input of size [N,C,H, W]
N==>batch size,
C==> number of channels,
H==> height of input planes in pixels,
W==> width in pixels.
'''

import torch
import torchvision
from PIL import Image
import torchvision.transforms as T

# Read input image
img = Image.open('dogncat.jpg')

# convert the input image to torch tensor
img = T.ToTensor()(img)
print("Input image size:", img.size()) # size = [3, 466, 700]

# unsqueeze the image to make it 4D tensor
img = img.unsqueeze(0) # image size = [1, 3, 466, 700]
# define convolution layer
# conv = nn.Conv2d(in_channels, out_channels, kernel_size)
conv = torch.nn.Conv2d(3, 3, 2)

# apply convolution operation on image
img = conv(img)
print(img.size())
plt.imshow(img[0,:,:,:].detach().numpy())


# squeeze image to make it 3D
img = img.squeeze(0) #now size is again [3, 466, 700]
# convert image to PIL image
img = T.ToPILImage()(img)

# display the image after convolution
img.show()

### Primary layer unit test

In [ ]:
def random_Tensor(size):
    return torch.rand(size, size, size)
a = [c for c in random_Tensor(3) ]
print(a)
a = torch.cat(a)
print(a)